### Calculate CBD to Suburb Distance

This notebook takes a suburb and calculates the distance from that suburb to the CBD.

### API Key!!
You need an Openroute API key to execute this notebook. Place it inside `data/metadata/openroute.json` with an attribute of key.

In [18]:
import pandas as pd
import geopandas as gpd
import openrouteservice
import sys
import os
from geopy.distance import geodesic
import time
import json

sys.path.append(os.path.abspath('../../scripts/2. modules'))
import processing

MELBOURNE_CENTRAL_COORD = (-37.810272, 144.962646)

# Output
OUTPUT_PATH = '../../data/2. raw'
OUTPUT_FILE_NAME = 'distances.csv'

OPENROUTE_API_KEY_PATH = '../../data/metadata/'

In [1]:
regions_df = processing.get_regions_df(2)

key = json.load(OPENROUTE_API_KEY_PATH)['key']

client = openrouteservice.Client(key=key)

regions_df['centroid'] = regions_df.geometry.centroid

crow_distances = []

# Iterate over the rows in the GeoDataFrame
for index, row in regions_df.iterrows():
    # Extract the latitude and longitude of the centroid
    centroid_coords = (row['centroid'].y, row['centroid'].x)  # y is latitude, x is longitude
    
    # Calculate the straight-line distance (Haversine distance)
    distance = geodesic(centroid_coords, MELBOURNE_CENTRAL_COORD).kilometers  # Distance in kilometers
    crow_distances.append(distance)

# Add the calculated distances to the GeoDataFrame
regions_df['crow_distance_to_cbd'] = crow_distances

NameError: name 'processing' is not defined

Calculate the driving distance to each place

In [13]:
gdf = regions_df

DELAY_TIME = 3

distances = []
durations = []
geometries = []

for index, row in gdf.iterrows():
    print("Downloading: ", index)
    centroid_coords = list(row['centroid'].coords[0])  # Get centroid coordinates
    mc_reversed = tuple(reversed(MELBOURNE_CENTRAL_COORD))

    # Request route between centroid and specific point
    route = client.directions(
        coordinates=[centroid_coords, mc_reversed],
        profile='driving-car',
        format='geojson',
        radiuses=[-1,350]
    )
    
    # Extract distance (in meters) and duration (in seconds)
    distance = route['features'][0]['properties']['segments'][0]['distance']
    duration = route['features'][0]['properties']['segments'][0]['duration']
    route_geometry = route['features'][0]['geometry']['coordinates']

    distances.append(distance)
    durations.append(duration)
    geometries.append(route_geometry)

    print("Downloaded successfully: ", index, distance, duration)

    time.sleep(DELAY_TIME)


Downloading:  0
Downloaded successfully:  0 6702.5 811.4
Downloading:  1
Downloaded successfully:  1 17936.7 1656.4
Downloading:  2
Downloaded successfully:  2 10138.0 1002.8
Downloading:  3
Downloaded successfully:  3 50846.3 2914.2
Downloading:  4
Downloaded successfully:  4 279492.3 12766.0
Downloading:  5
Downloaded successfully:  5 118540.7 5479.2
Downloading:  6
Downloaded successfully:  6 14247.7 1223.5
Downloading:  7
Downloaded successfully:  7 35745.3 2397.5
Downloading:  8
Downloaded successfully:  8 86743.0 4338.3
Downloading:  9
Downloaded successfully:  9 205707.0 9020.0
Downloading:  10
Downloaded successfully:  10 152319.8 7224.9
Downloading:  11
Downloaded successfully:  11 19814.9 1782.0
Downloading:  12
Downloaded successfully:  12 46869.6 2678.4
Downloading:  13
Downloaded successfully:  13 22893.0 1712.1
Downloading:  14
Downloaded successfully:  14 37291.8 2461.9
Downloading:  15
Downloaded successfully:  15 19821.7 1522.4
Downloading:  16
Downloaded successfully:

Save the data to a csv

In [19]:
gdf['distance_to_cbd'] = distances 
gdf['duration_to_cbd'] = durations
gdf['route_to_cbd'] = geometries

processing.to_csv(gdf, OUTPUT_PATH, OUTPUT_FILE_NAME)